In [1]:
import sys
import numpy as np
import pandas as pd
import re
import subprocess
from pathlib import Path
import fileinput


In [2]:
# ----------------------------------------------------------Anarci--------------------------------------------------

def anarci(seq,scheme):
    out =subprocess.run(['anarci', '--sequence', seq, '--scheme', scheme, '--assign_germline'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    output = out.stdout.decode("utf-8").splitlines()
    stats = {}
    hd = []
    anarci_dict = {'chain':[], 'new_residue_number':[], 'new_insertion':[]}
    for line_i, line in enumerate(output):
        l = line.strip()
        li = line.split()
        if line.startswith('#'):
            hspl= line.split('|')
            hd.append(hspl)
        else:
            if len(li) == 3:
                if li[2] == '-':
                    continue
                else:
                    anarci_dict['chain'].append(li[0])
                    anarci_dict['new_residue_number'].append(int(li[1]))
                    anarci_dict['new_insertion'].append('')
                    residue = li[2]
                    
            elif len(li) == 4:
                anarci_dict['chain'].append(li[0])
                anarci_dict['new_residue_number'].append(int(li[1]))
                anarci_dict['new_insertion'].append(str(li[2]))
                residue = li[2]   

    stats = {'chain' : hd[5][2], 'start': hd[5][5], 'stop': hd[5][6], 'species' : hd[8][1], 
            'v_gene' : hd[8][2], 'v_id' : hd[8][3], 'j_gene' : hd[8][4], 'j_id' : hd[8][5]}     
    df_renum = pd.DataFrame.from_dict(anarci_dict)
    return(stats, df_renum)

In [3]:
pdbfile='/home/drewaight/Desktop/hdd1/XGmAbBoost/daratumumab1/Daratumumab_M_1yh3_CD38/Daratumumab_M_1yh3_CD38.pdb'
scheme='aho'
outfile='/media/hdd1/XGmAbBoost/daratumumab2/test.pdb'

In [4]:
from biopandas.pdb import PandasPdb

ppdb = PandasPdb()
ppdb.read_pdb(pdbfile)

chains = ppdb.df['ATOM']['chain_id'].unique()
chain_num = len(chains)

df =ppdb.df['ATOM'].copy()
hdf = df[(df['chain_id'] == 'H')].reset_index(drop=True)
ldf = df[(df['chain_id'] == 'L')].reset_index(drop=True)
adf = df[(df['chain_id'] == 'A')].reset_index(drop=True)
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['chain_id'] == 'A']

def seq_order(df):
    from collections import OrderedDict
    df['residue_insertion'] = df['residue_number'].astype(str)+df['insertion'].astype(str)
    ordered_seq = list(OrderedDict.fromkeys(df['residue_insertion']))
    seq_dict = {ordered_seq[i]: i+1 for i in range(0, len(ordered_seq))}
    df['residue_insertion'] = df['residue_insertion'].map(seq_dict)
    df['residue_number'] = df['residue_insertion']
    df.drop(['residue_insertion'], axis=1, inplace = True)
    df['insertion'] = ''
    return(df)

In [5]:
df.tail()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
7298,ATOM,7301,,H,,CYS,,A,296,,...,45.784,-37.367,43.622,1.0,0.0,,,H,NaN,7316
7299,ATOM,7302,,HA,,CYS,,A,296,,...,45.111,-39.065,44.998,1.0,0.0,,,H,NaN,7317
7300,ATOM,7303,,1HB,,CYS,,A,296,,...,47.763,-40.458,44.458,1.0,0.0,,,H,NaN,7318
7301,ATOM,7304,,2HB,,CYS,,A,296,,...,46.435,-41.076,45.388,1.0,0.0,,,H,NaN,7319
7302,ATOM,7305,,HG,,CYS,,A,296,,...,44.663,-41.022,43.519,1.0,0.0,,,H,NaN,7320


In [6]:
hdf.describe()

,atom_number,residue_number,x_coord,y_coord,z_coord,occupancy,b_factor,charge,line_idx
count,1759.000000,1759.000000,1759.000000,1759.000000,1759.000000,1759.0,1759.000000,0.0,1759.000000
mean,880.000000,69.752700,33.751504,-9.616121,68.909403,1.0,0.100625,NaN,895.000000
std,507.923879,40.150112,8.691372,7.560993,7.675293,0.0,0.300918,NaN,507.923879
min,1.000000,1.000000,15.558000,-26.542000,46.785000,1.0,0.000000,NaN,16.000000
25%,440.500000,39.000000,26.693500,-15.457500,63.850500,1.0,0.000000,NaN,455.500000
50%,880.000000,73.000000,33.370000,-9.717000,69.380000,1.0,0.000000,NaN,895.000000
75%,1319.500000,100.000000,40.963500,-3.865000,74.759500,1.0,0.000000,NaN,1334.500000
max,1759.000000,144.000000,53.945000,7.964000,86.126000,1.0,1.000000,NaN,1774.000000


In [7]:
hdf = seq_order(hdf)
ldf = seq_order(ldf)
ppdb.df['ATOM'].drop(ppdb.df['ATOM'].index, inplace=True)
ppdb.df['ATOM'] = ppdb.df['ATOM'].append(hdf)
ppdb.df['ATOM'] = ppdb.df['ATOM'].append(ldf).reset_index(drop=True)

def df_to_seq(chain):
    seq = ppdb.amino3to1()
    seq = str(''.join(seq.loc[seq['chain_id'] == chain, 'residue_name']))
    return(seq)

In [8]:
hdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   record_name     1759 non-null   object 
 1   atom_number     1759 non-null   int64  
 2   blank_1         1759 non-null   object 
 3   atom_name       1759 non-null   object 
 4   alt_loc         1759 non-null   object 
 5   residue_name    1759 non-null   object 
 6   blank_2         1759 non-null   object 
 7   chain_id        1759 non-null   object 
 8   residue_number  1759 non-null   int64  
 9   insertion       1759 non-null   object 
 10  blank_3         1759 non-null   object 
 11  x_coord         1759 non-null   float64
 12  y_coord         1759 non-null   float64
 13  z_coord         1759 non-null   float64
 14  occupancy       1759 non-null   float64
 15  b_factor        1759 non-null   float64
 16  blank_4         1759 non-null   object 
 17  segment_id      1759 non-null   o

In [9]:
h_seq = df_to_seq('H')
l_seq = df_to_seq('L')

In [10]:
h_seq

'EVQLLESGGGLVQPGGSLRLSCAVSGFTFNSFAMSWVRQAPGKGLEWVSAISGSGGGTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYFCAKDKILWFGEPVFDYWGQGTL'

In [12]:
# ------------------------------------------------BioPandas Cleanup--------------------------------------------------

hdf_stats, hdf_renum = anarci(h_seq, scheme)
hdf_renum.drop(['chain'], inplace = True, axis=1)
hdf_renum['residue_number'] = np.arange(len(hdf_renum)) + 1
hdf.drop(hdf[hdf['residue_number'] > int(hdf_stats['stop'])].index, inplace=True)
hdf = hdf.merge(hdf_renum, how='left') \
.drop(columns=['residue_number', 'insertion']) \
.rename(columns={'new_residue_number': 'residue_number','new_insertion': 'insertion'})

ldf_stats, ldf_renum = anarci(l_seq, scheme)
ldf_renum.drop(['chain'], inplace = True, axis=1)
ldf_renum['residue_number'] = np.arange(len(ldf_renum)) + 1
ldf.drop(ldf[ldf['residue_number'] > int(ldf_stats['stop'])].index, inplace=True)
ldf = ldf.merge(ldf_renum, how='left') \
.drop(columns=['residue_number', 'insertion']) \
.rename(columns={'new_residue_number': 'residue_number','new_insertion': 'insertion'})

def atom_renum(df):
    from collections import OrderedDict
    atom_list = list(OrderedDict.fromkeys(df['atom_number']))
    atom_dict = {atom_list[i]: i+1 for i in range(0, len(atom_list))}
    df['atom_number'] = df['atom_number'].map(atom_dict)
    return(df)

ppdb.df['ATOM'].drop(ppdb.df['ATOM'].index, inplace=True)
hdf = hdf.append(ldf, sort=False)
hdf = hdf.append(adf, sort=False)
hdf = atom_renum(hdf).reset_index(drop=True)
ppdb.df['ATOM'] = ppdb.df['ATOM'].append(hdf, sort=False).reset_index(drop=True)
ppdb.to_pdb(outfile, records=['ATOM'])
# print(ppdb.df['ATOM'].head())

h_stats = pd.DataFrame.from_records([hdf_stats])
l_stats = pd.DataFrame.from_records([ldf_stats])

h_stats, l_stats

(  chain start stop species        v_gene  v_id    j_gene  j_id
 0     H     0  116   human  IGHV3-23D*01  0.95  IGHJ4*01  0.64,
   chain start stop species       v_gene  v_id    j_gene  j_id
 0     K     0  100   human  IGKV3-11*01  0.98  IGKJ2*01  0.58)